In [1]:
import numpy as np
import pandas as pd
import math
import random
import csv

import pickle

from params import *
from functions import *

import os
os.makedirs('results/', exist_ok = False) 

In [2]:
for SEED in range(0,10):
    
    lambda_results = {}

    set_seed(SEED)
    data_hist = pd.read_pickle(f'data/data_hist_{SEED}.pickle')

    pred_prob_original = np.zeros((N_hist,len(passenger_incentive_list),len(driver_incentive_list)))
    fare_list = np.zeros(N_hist)
    compensation_list = np.zeros(N_hist)

    for n in range(0,N_hist):

        fare_list[n] = data_hist.loc[n, 'fare']
        compensation_list[n] = data_hist.loc[n, 'compensation']

        for i in range(0, len(passenger_incentive_list)):
            for j in range(0, len(driver_incentive_list)):
                pred_prob_original[n,i,j] = data_hist.loc[n, f'pred_prob_{i}_{j}']

    # obtain the results under different proportions of passenger-side budget
    for passenger_percentage in [0, 0.2, 0.4, 0.6, 0.8, 1]:
        
        if (passenger_percentage == 0):
            lambda_1 = np.inf
            lambda_2 = solve_largrange_multiplier_only_driver(N_hist, pred_prob_original, fare_list, compensation_list, passenger_incentive_list, driver_incentive_list, eta=eta, B=B_hist)

        elif (passenger_percentage == 1):
            lambda_2 = np.inf
            lambda_1 = solve_largrange_multiplier_only_passenger(N_hist, pred_prob_original, fare_list, compensation_list, passenger_incentive_list, driver_incentive_list, eta=eta, B=B_hist)

        else:
            lambda_1, lambda_2 = solve_largrange_multiplier_sep(N_hist, pred_prob_original, fare_list, compensation_list, passenger_incentive_list, driver_incentive_list, eta=eta, B=B_hist, passenger_percentage=passenger_percentage)
    
        print(SEED, passenger_percentage, lambda_1, lambda_2)
        lambda_results[passenger_percentage] = [lambda_1, lambda_2]
    
    pickle.dump(lambda_results, open(f'results/results_offline_SEP_{SEED}.pickle', 'wb'))

Finish solving Lagrange multiplier, the constraint function is: 2990.6323771891757
0 0 inf 0.039162467957269576
Finish solving Lagrange multiplier, the constraint function is: 597.8832504434127 2394.929111830234
0 0.2 0.051281773311624576 0.04481873627912736
Finish solving Lagrange multiplier, the constraint function is: 1200.7317549226789 1794.4726638122502
0 0.4 0.035586995714472575 0.052488252647748077
Finish solving Lagrange multiplier, the constraint function is: 1803.0064871764273 1200.2995700720635
0 0.6 0.033835020540836994 0.05766517619656705
Finish solving Lagrange multiplier, the constraint function is: 2401.2858867425266 600.7496081120441
0 0.8 0.030915061918111043 0.07540093279900248
Finish solving Lagrange multiplier, the constraint function is: 3007.5111764404946
0 1 0.026827119846294708 inf
Finish solving Lagrange multiplier, the constraint function is: 2992.314789491776
1 0 inf 0.038865091653626185
Finish solving Lagrange multiplier, the constraint function is: 599.909